In [1]:
import numpy as np
import math

Load csv file as np.array

In [2]:
def load_data():
    with open('pima-indians-diabetes.csv') as f: 
        raw_data = np.array([line.split(",")[:-1] for line in f])
        columns = raw_data[0]
        data = raw_data[1:]
    return columns,data.astype(float)

Split dataset according to holdout validation

In [3]:
def test_train_split(data,fraction):
    n_entries = len(data)
    n_train = int(fraction*n_entries)
    
    indicies = np.random.permutation(np.arange(n_entries))

    train_set = data[indicies[:n_train]]
    test_set = data[indicies[n_train:]]
    
    return train_set, test_set

Calculate mean and std for each binary class

In [106]:
class NaiveBayesPredictor():
    
    def __init__(self,data):
        self.data = data
        self.calculate_mean_and_std()
        
    def predict(self,values):
        return self.calc_probabilities(values,0)
    
    def calculate_mean_and_std(self):
        class_0 = self.data[self.data[:,-1] == 0]
        class_1 = self.data[self.data[:,-1] == 1]
    
        self.mean = [np.mean(class_0, axis=0)[:-1], np.mean(class_1, axis=0)[:-1]]
        self.std = [np.std(class_0, axis=0)[:-1], np.std(class_1, axis=0)[:-1]] 
    
    def calc_probabilities(self,x):
        
        exponent = np.exp( -np.power(x-self.mean[0],2) / np.power(2*self.std[0],2) )
        return (1 / (math.sqrt(2*math.pi) * self.std[0])) * exponent
    

In [107]:
c = NaiveBayesPredictor(data)

array([1, 2, 3, 4, 5, 6, 7, 8])

In [108]:
c.calc_probabilities(data[1])

ValueError: operands could not be broadcast together with shapes (9,) (8,) 

In [44]:
data[1]

array([  1.        ,   0.42713568,   0.54098361,   0.29292929,
         0.        ,   0.39642325,   0.11656704,  31.        ,   0.        ])

In [4]:
columns, data = load_data()

In [5]:
train_set,test_set = test_train_split(data,0.95)